In [41]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import datetime

In [42]:
conn = sqlite3.connect("db.sqlite3")
cursor = conn.cursor()

In [43]:
BASE_URL = 'https://www.agronegocios.co'

In [44]:
def obtener_html(url):
  """Scrapes a website and extracts data.

  Args:
    url: The URL of the website to scrape.

  Returns:
    A list of data extracted from the website.
  """

  headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
        'Accept-Language': 'es-ES,es;q=0.9'
    }
  try:
      response = requests.get(url, headers=headers, timeout=10)
      response.raise_for_status()
      return response.text
  except requests.exceptions.RequestException as e:
      print(f"Error al obtener {url}: {str(e)}")
      return None

In [45]:
def extraer_autor(url_noticia):
    """Extrae el nombre del autor desde la URL de una noticia."""
    html = obtener_html(url_noticia)
    if not html:
        return "Desconocido"

    soup = BeautifulSoup(html, 'html.parser')
    autor_span = soup.select_one('.author-save .author span')
    autor_a = soup.select_one('.author-save .author a')
    return autor_span.text.strip() if autor_span else autor_a.text.strip() if autor_a else "Desconocido"

In [46]:
def extraer_noticias():
    """Extrae noticias desde la página principal."""
    html = obtener_html(BASE_URL)
    if not html:
        print("No se pudo obtener el HTML de la página.")
        return

    soup = BeautifulSoup(html, 'html.parser')
    articulos = soup.find_all('div', class_='post-card-1')

    if not articulos:
        print("⚠ No se encontraron artículos en la página.")
        return

    for i, articulo in enumerate(articulos):
        if i >= 8:
            break
        
        try:
            category = articulo.find('span').text.strip() if articulo.find('span') else "Sin categoría"
            title = articulo.find('h3').text.strip() if articulo.find('h3') else "Sin título"
            image = articulo.find('img')['data-src'] if articulo.find('img') and 'data-src' in articulo.find('img').attrs else ""
            relative_url = articulo.find('a')['href'] if articulo.find('a') else "#"
            new_url = f"{BASE_URL}{relative_url}"
            extracted_at = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            author = extraer_autor(new_url)

            noticia = {
                'Categoría': category,
                'Título': title,
                'Imagen URL': image,
                'URL Noticia': new_url,
                'Autor': author
            }

            cursor.execute("INSERT INTO news (category, title, image_url, news_url, author, extracted_at) VALUES (?, ?, ?, ?, ?, ?)", (category, title, image, new_url, author, extracted_at))
            conn.commit()

            print(noticia)

        except Exception as e:
            print(f"Error procesando noticia: {str(e)}")

In [47]:
extraer_noticias()

{'Categoría': 'Porcinos', 'Título': 'El avance que cambiaría el control de los virus porcinos', 'Imagen URL': 'https://img.lalr.co/cms/2020/09/10094902/cerdos.png?r=16_9', 'URL Noticia': 'https://www.agronegocios.co/finca/la-interaccion-de-la-gripe-por-porcina-y-el-prrsv-cambiaria-el-control-de-los-virus-en-cerdos-4069087', 'Autor': 'Juliana Ramos Mosquera'}
{'Categoría': 'Agricultura', 'Título': 'Empezó el Salón Internacional de la Agricultura en París, Francia', 'Imagen URL': 'https://img.lalr.co/cms/2025/02/14130429/Bovinos-4.jpg?r=16_9', 'URL Noticia': 'https://www.agronegocios.co/agricultura/se-reunen-agricultores-del-mundo-para-el-salon-internacional-de-agricultura-en-paris-4068751', 'Autor': 'Juliana Ramos Mosquera'}
{'Categoría': 'Agricultura', 'Título': 'Caída del cacao: preocupaciones por demanda y suministros', 'Imagen URL': 'https://img.lalr.co/cms/2025/01/13100154/Cacao-seco.jpg?r=16_9', 'URL Noticia': 'https://www.agronegocios.co/agricultura/caida-del-cacao-preocupaciones

In [48]:
conn.close()